In [ ]:
# Googleドライブへのマウント（Colab用コード）
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/ou_dip/"

In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

def imshow(img):
  if img.ndim == 3:
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    display(Image.fromarray(img))
  else:
    display(Image.fromarray(img))

In [ ]:
# 入力画像
img = cv2.imread("sample.jpg")

In [ ]:
# superpixel segmentation (SLIC)
# https://docs.opencv.org/master/df/d6c/group__ximgproc__superpixel.html
# image	Image to segment
# algorithm	Chooses the algorithm variant to use: SLIC segments image using a desired region_size, and in addition SLICO will optimize using adaptive compactness factor, while MSLIC will optimize using manifold methods resulting in more content-sensitive superpixels.
# region_size	Chooses an average superpixel size measured in pixels
# ruler	Chooses the enforcement of superpixel smoothness factor of superpixel

lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
slic = cv2.ximgproc.createSuperpixelSLIC(lab, cv2.ximgproc.SLICO, region_size=30, ruler=10.0)
slic.iterate(4)
slic.enforceLabelConnectivity(min_element_size=10)
contours_dst = img.copy()

# contour mask 
contours = slic.getLabelContourMask(False)
contours_dst[0 < contours] = (0, 255, 255)
print("SLIC contours")
imshow(contours_dst)

# labels
superpixels = slic.getLabels()
print("SLIC masks")
plt.figure()
plt.imshow(superpixels)
#imshow(superpixels.astype(np.uint8))

In [ ]:
# 領域がつながっているか判定

def is_connected(mask, i, j):
  mask_i = (mask==i).astype(np.uint8)
  mask_j = (mask==j).astype(np.uint8)

  kernel8 = np.ones((3,3),np.uint8)
  mask_i_dilated = cv2.dilate(mask_i,kernel8,iterations = 1)
  
  return np.sum(mask_i_dilated & mask_j) > 0

In [ ]:
# Superpixelを初期値にした region unification (てきとう実装；時間かかる)

mask = superpixels.copy()
num_regions = np.max(mask)
threshold = 40.0

for k in range(1):  # 統合1回だけにした

  # 特徴：領域中のLABの平均
  mean_lab = []
  for i in range(num_regions):
    # features
    mean_lab.append(np.mean(lab[mask==i],axis=0))

  # 隣接領域で、一番特徴が近いものを選択
  min_distance = np.ones(num_regions)*threshold
  min_id = np.array(range(num_regions))

  for i in range(num_regions):
    for j in range(num_regions):
      if j<=i: continue

      if is_connected(mask,i,j):
        d = np.linalg.norm(mean_lab[i]-mean_lab[j])
        if(min_distance[i] > d):
          min_distance[i] = d
          min_id[i] = j
    
  #print(min_id,min_distance)
  
  # ラベルを更新
  for i in reversed(range(num_regions)):
    #print(i,min_id[i])
    j = min_id[i]
    if j < 0: continue

    min_id[j] = i
    min_id = np.where(min_id==j,i,min_id)

  for i in range(num_regions):
    mask = np.where(mask==i,min_id[i],mask)

#mask = mask.astype(np.uint8)
#imshow(mask)

plt.figure()
plt.imshow(mask)